In [1]:
import streamlit as st
import jsonlines
import random
import pandas as pd
import json
import fileinput
import numpy as np

2022-05-21 10:45:40.238 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [89]:
#Importing Dataset
olafScholzJsonLines = jsonlines.open("OlafScholz.jl")
olafScholzTwitter = pd.read_json("OlafScholz.jl", lines = True)
print("Datensatzlänge: " + str(olafScholzTwitter.info()))

print(olafScholzTwitter.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   http_status   65 non-null     int64 
 1   account_name  65 non-null     object
 2   account_data  65 non-null     object
 3   params        65 non-null     object
 4   response      65 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.7+ KB
Datensatzlänge: None


,http_status,account_name,account_data,params,response
0,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'context_annotations': [{'domain': ...
1,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...","{'data': [{'in_reply_to_user_id': '38150247', ..."
2,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'conversation_id': '144576374098134...
3,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'created_at': '2021-09-02T13:33:12....
4,200,OlafScholz,"{'Name': 'Scholz, Olaf', 'Partei': 'SPD', 'id'...","[[query, from:OlafScholz], [max_results, 100],...",{'data': [{'conversation_id': '141999599421797...


In [86]:
print("Response Metadaten: " + str(olafScholzTwitter["response"][0].keys()))
ResponseKeys = olafScholzTwitter["response"][0].keys()
print(ResponseKeys)
for Key in ResponseKeys:
    print("{}-Metadaten: ".format(Key) + str(olafScholzTwitter["response"][0][Key]))

Response Metadaten: dict_keys(['data', 'includes', 'meta'])
dict_keys(['data', 'includes', 'meta'])
data-Metadaten: [{'context_annotations': [{'domain': {'id': '10', 'name': 'Person', 'description': 'Named people in the world like Nelson Mandela'}, 'entity': {'id': '1439961464555466753', 'name': 'Olaf Scholz'}}, {'domain': {'id': '29', 'name': 'Events [Entity Service]', 'description': 'Entity Service related Events domain'}, 'entity': {'id': '1372917575357591554', 'name': 'Easter', 'description': 'this entity for all conversations for Easter all year '}}, {'domain': {'id': '35', 'name': 'Politician', 'description': 'Politicians in the world, like Joe Biden'}, 'entity': {'id': '1439961464555466753', 'name': 'Olaf Scholz'}}, {'domain': {'id': '119', 'name': 'Holiday', 'description': 'Holidays like Christmas or Halloween'}, 'entity': {'id': '1372917575357591554', 'name': 'Easter', 'description': 'this entity for all conversations for Easter all year '}}, {'domain': {'id': '123', 'name': '

In [70]:
#print(olafScholzTwitter["response"][0].len)
print("---")
#Starting to investigate Dataset
#print(olafScholzTwitter.describe())
print(type(olafScholzTwitter))
print(olafScholzTwitter.keys())
for key in olafScholzTwitter:
    #print("{} : {}".format(key, olafScholzTwitter[key]))
    print("{}".format(key) + str(type(olafScholzTwitter[key])))
    

---
<class 'pandas.core.frame.DataFrame'>
Index(['http_status', 'account_name', 'account_data', 'params', 'response'], dtype='object')
http_status<class 'pandas.core.series.Series'>
account_name<class 'pandas.core.series.Series'>
account_data<class 'pandas.core.series.Series'>
params<class 'pandas.core.series.Series'>
response<class 'pandas.core.series.Series'>
